In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2' #显示log的详细程度 0 为最详细 取： 0 1 2
import tensorflow as tf
import numpy as np
import gym
from gym.spaces import Discrete, Box

In [3]:
env_name = "CartPole-v0"
env = gym.make(env_name)

In [5]:
obs_dim = env.observation_space.shape[0]
obs_dim

4

In [6]:
n_acts = env.action_space.n
n_acts

2

In [17]:
obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
obs_ph

<tf.Tensor 'Placeholder_1:0' shape=(?, 4) dtype=float32>

In [8]:
def mlp(x, sizes, activation=tf.tanh, output_activation=None):
    # Build a feedforward neural network.
    for size in sizes[:-1]:
        x = tf.layers.dense(x, units=size, activation=activation)
    return tf.layers.dense(x, units=sizes[-1], activation=output_activation)

In [19]:
hidden_sizes=[32]
logits = mlp(obs_ph, sizes=c)  # input:4  hidden layer:32  output:2

In [20]:
hidden_sizes+[n_acts]

[32, 2]

In [22]:
# make action selection op (outputs int actions, sampled from policy)
actions = tf.squeeze(tf.multinomial(logits=logits,num_samples=1), axis=1)
actions

<tf.Tensor 'Squeeze_1:0' shape=(?,) dtype=int64>

In [23]:
weights_ph = tf.placeholder(shape=(None,), dtype=tf.float32)

In [24]:
act_ph = tf.placeholder(shape=(None,), dtype=tf.int32)
action_masks = tf.one_hot(act_ph, n_acts)
log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logits), axis=1)
loss = -tf.reduce_mean(weights_ph * log_probs)

In [33]:
action_masks

<tf.Tensor 'one_hot:0' shape=(?, 2) dtype=float32>

In [26]:
lr = 1e-2
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [27]:

    # for training policy
    def train_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            act = sess.run(actions, {obs_ph: obs.reshape(1,-1)})[0]
            obs, rew, done, _ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []

                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break
                            # take a single policy gradient update step
        batch_loss, _ = sess.run([loss, train_op],
                                 feed_dict={
                                    obs_ph: np.array(batch_obs),
                                    act_ph: np.array(batch_acts),
                                    weights_ph: np.array(batch_weights)
                                 })
        return batch_loss, batch_rets, batch_lens

In [34]:
epochs = 30
render = "store_true"
batch_size = 5000
for i in range(epochs):
    batch_loss, batch_rets, batch_lens = train_one_epoch()
    print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
            (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))

epoch:   0 	 loss: 99.923 	 return: 200.000 	 ep_len: 200.000
epoch:   1 	 loss: 98.348 	 return: 197.115 	 ep_len: 197.115
epoch:   2 	 loss: 98.417 	 return: 200.000 	 ep_len: 200.000
epoch:   3 	 loss: 97.963 	 return: 195.885 	 ep_len: 195.885
epoch:   4 	 loss: 98.509 	 return: 200.000 	 ep_len: 200.000
epoch:   5 	 loss: 99.114 	 return: 197.038 	 ep_len: 197.038
epoch:   6 	 loss: 94.741 	 return: 188.148 	 ep_len: 188.148
epoch:   7 	 loss: 99.342 	 return: 200.000 	 ep_len: 200.000
epoch:   8 	 loss: 99.060 	 return: 200.000 	 ep_len: 200.000
epoch:   9 	 loss: 98.611 	 return: 195.885 	 ep_len: 195.885
epoch:  10 	 loss: 98.903 	 return: 199.308 	 ep_len: 199.308
epoch:  11 	 loss: 97.139 	 return: 187.741 	 ep_len: 187.741
epoch:  12 	 loss: 99.644 	 return: 195.808 	 ep_len: 195.808
epoch:  13 	 loss: 96.779 	 return: 194.000 	 ep_len: 194.000
epoch:  14 	 loss: 98.422 	 return: 195.500 	 ep_len: 195.500
epoch:  15 	 loss: 99.275 	 return: 198.231 	 ep_len: 198.231
epoch:  